# Oceananigans (Flat, Bounded, Bounded)

Currently not working 

https://github.com/CliMA/Oceananigans.jl/issues/1023

In [1]:
versioninfo()

Julia Version 1.5.2
Commit 539f3ce943 (2020-09-23 23:17 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i7-4980HQ CPU @ 2.80GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, haswell)


In [2]:
] status

Status `~/.julia/environments/v1.5/Project.toml`
  [c52e3926] Atom v0.12.24
  [6e4b80f9] BenchmarkTools v0.5.0
  [aae7a2af] DiffEqFlux v1.24.0
  [0c46a032] DifferentialEquations v6.15.0
  [587475ba] Flux v0.11.1
  [7073ff75] IJulia v1.22.0
  [033835bb] JLD2 v0.2.4
  [e5e0dc1b] Juno v0.8.4
  [19f0c83c] NavierStokes2D v0.1.0 `~/.julia/dev/NavierStokes2D`
  [9e8cae18] Oceananigans v0.44.0
  [14b8a8f1] PkgTemplates v0.7.13
  [91a5bcdd] Plots v1.9.0
  [90137ffa] StaticArrays v0.12.4


In [3]:
using Oceananigans, Oceananigans.Advection, Oceananigans.Fields, Oceananigans.Grids

In [4]:
grid2Dtrue = RegularCartesianGrid(topology = (Flat, Bounded, Bounded),
                              size=(64, 64), 
                              extent=(2π, 2π))

RegularCartesianGrid{Float64, Flat, Bounded, Bounded}
                   domain: x ∈ [0.0, 0.0], y ∈ [8.410780489584512e-18, 6.283185307179587], z ∈ [-6.283185307179586, 5.517472001167444e-16]
                 topology: (Flat, Bounded, Bounded)
  resolution (Nx, Ny, Nz): (1, 64, 64)
   halo size (Hx, Hy, Hz): (0, 1, 1)
grid spacing (Δx, Δy, Δz): (0.0, 0.09817477042468103, 0.09817477042468103)

In [5]:
v_bcs = VVelocityBoundaryConditions(grid2Dtrue,
           top = ValueBoundaryCondition(1.0),
        bottom = ValueBoundaryCondition(0.0))

w_bcs = WVelocityBoundaryConditions(grid2Dtrue,
        north = ValueBoundaryCondition(0.0),
        south = ValueBoundaryCondition(0.0))

Oceananigans.FieldBoundaryConditions (NamedTuple{(:x, :y, :z)}), with boundary conditions
├── x: CoordinateBoundaryConditions{Nothing,Nothing}
├── y: CoordinateBoundaryConditions{BoundaryCondition{Value,Float64},BoundaryCondition{Value,Float64}}
└── z: CoordinateBoundaryConditions{BoundaryCondition{NormalFlow,Nothing},BoundaryCondition{NormalFlow,Nothing}}

In [6]:
model2Dtrue = IncompressibleModel(grid = grid2Dtrue,
                            timestepper = :RungeKutta3,
                             advection = WENO5(),
                               #buoyancy = BuoyancyTracer(),
                               # tracers = :b,
                              buoyancy = nothing,
                                tracers = nothing,
                    boundary_conditions = (v=v_bcs, w=w_bcs),
                                closure = IsotropicDiffusivity(ν=2e-4, κ=2e-4)
                           )

IncompressibleModel{CPU, Float64}(time = 0 seconds, iteration = 0) 
├── grid: RegularCartesianGrid{Float64, Flat, Bounded, Bounded}(Nx=1, Ny=64, Nz=64)
├── tracers: ()
├── closure: IsotropicDiffusivity{Float64,NamedTuple{(),Tuple{}}}
├── buoyancy: Nothing
└── coriolis: Nothing

In [10]:
u, v, w = model2Dtrue.velocities;

Note that this will error:

`MethodError: no method matching bctype_str(::Nothing)`

if show is not surpressed, due to:
https://github.com/CliMA/Oceananigans.jl/issues/1026

In [13]:
progress(sim) = @info "Iteration: $(sim.model.clock.iteration), time: $(round(Int, sim.model.clock.time))"

simulation = Simulation(model2Dtrue, Δt=0.2, 
    stop_time=20.0, iteration_interval=20, progress=progress);

using Oceananigans.OutputWriters

simulation.output_writers[:fields] = 
        JLD2OutputWriter(model2Dtrue, 
                         (v=v,w=w),
                          schedule = TimeInterval(1.0),
                          prefix = "gravity_current",
                          force = true);

In [14]:
run!(simulation)

┌ Info: Iteration: 20, time: 4
└ @ Main In[13]:1
┌ Info: Iteration: 40, time: 8
└ @ Main In[13]:1
┌ Info: Iteration: 60, time: 12
└ @ Main In[13]:1
┌ Info: Iteration: 80, time: 16
└ @ Main In[13]:1
┌ Info: Iteration: 100, time: 20
└ @ Main In[13]:1
┌ Info: Simulation is stopping. Model time 20.000 seconds has hit or exceeded simulation stop time 20 seconds.
└ @ Oceananigans.Simulations /Users/williamdavis/.julia/packages/Oceananigans/IbUoB/src/Simulations/run.jl:40


In [15]:
using JLD2
file = jldopen(simulation.output_writers[:fields].filepath)
iterations = parse.(Int, keys(file["timeseries/t"]))
iterations'

1×21 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 0  5  11  15  20  26  31  36  41  46  …  60  65  70  75  80  85  90  95  100

In [16]:
file["timeseries/w/5"][1, :, :]

64×65 Array{Float32,2}:
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  0.0
 0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN 